In [1]:
import pandas as pd
from datetime import datetime
import re

In [2]:
# input the file of the dataset
file_path = 'BiotechCropsAllTables2024.xlsx'
xls = pd.ExcelFile(file_path)

# List to store the unified data
data = []

In [3]:
# Iterate over each sheet
for sheet_name in xls.sheet_names:
    df = pd.read_excel(xls, sheet_name=sheet_name, header=None)
    
    # Find the rows where each table starts
    table_starts = df[df.apply(lambda row: row.str.contains('State/Year', na=False).any(), axis=1)].index
    
    for start in table_starts:
        # Get and clean the gene name
        gene = df.iloc[start-1, 0].split('(')[0].strip()
        
        # Get and clean the years
        years = df.iloc[start, 1:].values
        years = [str(year).strip() for year in years if pd.notna(year)]
        try:
            years = [int(float(year)) for year in years]
        except ValueError:
            continue  # Skip if conversion fails
        
        # Get the states and adoption percentages
        table_data = df.iloc[start+1:]
        for i, row in table_data.iterrows():
            state = str(row[0])
            
            # Skip empty or invalid rows
            if pd.isna(state) or state.strip() == "":
                break

            # Clean the state name by removing disclaimers "X/X"
            if state.strip() != "United States":
                state = re.split(r'[\s]', state)[0].strip()
            else:
                state = state.strip()

            # Extract adoption percentages
            adoption_percentages = row[1:].values
            #adoption_percentages = int(adoption_percentages)
            for year, adoption_percentage in zip(years, adoption_percentages):
                if pd.notna(adoption_percentage):
                    data.append({
                        'gene': gene,
                        'state': state,
                        'crop': sheet_name,
                        'year': year,
                        'adoption_percentage': adoption_percentage
                    })

In [4]:
# Convert the list into a DataFrame
result_df = pd.DataFrame(data)z

# Adjust, drop the "State/Year" items of State col
result_df = result_df[result_df["state"]!='State/Year']

# Adjust, non numeric values in percentage col
result_df['adoption_percentage'] = pd.to_numeric(result_df['adoption_percentage'], errors='coerce')/100

# Save the result to a new Excel file
result_df.to_excel('output.xlsx', index=False)